Word Vector : Skip-gram Model
=============

With this project, the words are translated into vector embedding using skip-gram neural network model, in turn capturing the semantic and syntactic  knowledge of the word in the process. Semantic : house ~ apartment. Semantic : Puppy is to dog, what Kitten is to cat. 
The Word2Vec skip-gram model if generated in this project over [Text8](http://mattmahoney.net/dc/textdata) data.

**REF:To Know more on the skip-gram models , the theoritical foundation please visit [Skip-gram Word2Vector Models](https://deepdatascience.wordpress.com/2017/04/22/word2vec-skip-gram-model/),
**

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE
import datetime

STEP 1 : Download Text data
----------------------------
Download the text data from the source website.

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)
print('Completed', datetime.datetime.now() )

Found and verified text8.zip
Completed 2017-05-09 07:41:31.418077


**Step 2 :Tokenize text data into words.**
---------------------

In [3]:
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data
  
words = read_data('text8_mid_30MB.zip')
print('Data size in number of words %d' % len(words))
print('Completed', datetime.datetime.now() )

Data size in number of words 5094672
Completed 2017-05-09 07:41:32.112716


In [4]:
print(type(words))
print(words[0:10])
print( " ".join(words[0:100]))

<type 'list'>
['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']
anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing


Step 3: Build the  word  dictionary  i.e word - index 
------------------------------------------
1. Dictionary :  word - index dictionary as in  { word1 : 1, word2 : 2 }
2. Reverse_dictionary :  contains all words with key as index and value as word, so that it's easy to reference them e.g { word1 : 1, word2 : 2 } to reverse_dictionary becomes
{  1: word1 , 2: word2  }


In [5]:
vocabulary_size = 50000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Most common words (+UNK) [['UNK', 97870], ('the', 325643), ('of', 182950), ('and', 127938), ('one', 116440)]
Sample data [3181, 2980, 12, 6, 181, 2, 3736, 49, 58, 149]


In [6]:
n_pairs_dict = {k : dictionary[k] for k in dictionary.keys()[:10]}
print('dictionary - ', n_pairs_dict )

n_pairs_reverse_dict = {k : reverse_dictionary[k] for k in reverse_dictionary.keys()[:10]}
print('reverse dictionary - ',n_pairs_reverse_dict)


dictionary -  {'fawn': 42300, 'homomorphism': 15286, 'nunnery': 29778, 'chthonic': 31253, 'sonja': 42301, 'woods': 6307, 'clotted': 42302, 'spiders': 13019, 'gai': 34017, 'hanging': 7083}
reverse dictionary -  {0: 'UNK', 1: 'the', 2: 'of', 3: 'and', 4: 'one', 5: 'in', 6: 'a', 7: 'to', 8: 'zero', 9: 'nine'}


Step 4:  Morph word frequency into Skip-gram data
------------------------------
Skip gram model aims to predict the n-context words given a input target word. And in doins so, the nice side effect is that the word vectors are represented in an meaningful way capturing semantic and syntactic  relevance. 
In our case, since we have removed the commonly occuring words, our skip-gram model will  capture semantic similarity better and lesser of the syntactic relevance.


**skip_window** = 1 # How many words to consider left and right.

**num_skips** = 2 # How many times to reuse an input to generate a label.



In [7]:
print(type(data))
print(data[1:10])
print(reverse_dictionary[2307])
print(dictionary['originated']  )

<type 'list'>
[2980, 12, 6, 181, 2, 3736, 49, 58, 149]
ohio
2980


In [8]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window, reverse_dictionary = None , verbose = 0):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):    
    if verbose ==1:print('appending to buffer ',reverse_dictionary[data[data_index]])
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  if verbose ==1:print('iteration from 0 to ', batch_size // num_skips - 1,' inclusive' )
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer    
    targets_to_avoid = [ skip_window ]
    if verbose ==1:print('targets to avoid is skip_window ', targets_to_avoid)
    if verbose ==1:print('iterating from 0 to ', num_skips -1 ,' inclusive')
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
        if verbose ==1:print('getting target  word  randomly between 0 and ',span-1,' but not in the targets to avoid list', targets_to_avoid)
      targets_to_avoid.append(target)
      if verbose ==1:print('now add word ', reverse_dictionary[target], ' to targets_to_avoid list. After addition it is ',targets_to_avoid)
      batch[i * num_skips + j] = buffer[skip_window]      
      labels[i * num_skips + j, 0] = buffer[target]
      if verbose ==1:print( 'Adding word to batch , adding word to label  ',\
            reverse_dictionary[batch[i * num_skips + j]], reverse_dictionary[labels[i * num_skips + j, 0]] )
    buffer.append(data[data_index])
    if verbose ==1:print('adding ',reverse_dictionary[data[data_index]], ' to buffer ')
    data_index = (data_index + 1) % len(data)
    if verbose ==1:print('new data_index now is ', reverse_dictionary[data_index], ' from (data_index + 1) % len(data)' ,(data_index + 1) , len(data) )
    if verbose ==1:print(batch, labels)
  return batch, labels

print('Generating Train / Test Data from the text.')
print('Sample of first - 8 words data')
print(' '.join([reverse_dictionary[di] for di in data[:20]]) )

for num_skips, skip_window in [(1 , 1) , (1 , 2), (2, 1), (2,2) ,  (4, 2)]:
    data_index = 0
    _batch_size = 8
    #_batch_size = 12
    batch, labels = generate_batch(batch_size=_batch_size, num_skips=num_skips, skip_window=skip_window,\
                                   reverse_dictionary = reverse_dictionary)
    print('\nwith number of words to skip = %d and skip_window = %d:' % (num_skips, skip_window))        
    train_word_list = [reverse_dictionary[bi] for bi in batch] 
    label_word_list = [reverse_dictionary[li] for li in labels.reshape(_batch_size)]
    print('train_data : label is ', zip (train_word_list , label_word_list ) )

Generating Train / Test Data from the text.
Sample of first - 8 words data
anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english

with number of words to skip = 1 and skip_window = 1:
train_data : label is  [('originated', 'as'), ('as', 'a'), ('a', 'term'), ('term', 'of'), ('of', 'abuse'), ('abuse', 'first'), ('first', 'abuse'), ('used', 'first')]

with number of words to skip = 1 and skip_window = 2:
train_data : label is  [('as', 'term'), ('a', 'originated'), ('term', 'abuse'), ('of', 'abuse'), ('abuse', 'of'), ('first', 'against'), ('used', 'abuse'), ('against', 'early')]

with number of words to skip = 2 and skip_window = 1:
train_data : label is  [('originated', 'anarchism'), ('originated', 'as'), ('as', 'originated'), ('as', 'a'), ('a', 'term'), ('a', 'as'), ('term', 'of'), ('term', 'a')]

with number of words to skip = 2 and skip_window = 2:
train_data : label is  [('as', 'originated'), ('as', 'anarchism'), (

Code Analysis : 
--------------
word_window_size = 2 *  skip_window  + 1 
i.e skip_window  @ 1  =  w-1 w w +1 
i.e skip_window  @ 2  =  w-2 w-1 w w +1 w+2
i.e skip_window  @ 3  =  w-3 w-2 w-1 w w +1 w+2 w+3

num_skips = how many words can be skipped
i.e num_skips @ 1 = w-2 w w+2 , w-1 w w+1


In the code above, we have designed the   train data and labels, on the proximity basis. i.e when we are looking for 1 word near proximity i.e skip_window = 1,  for the word "originated" nearest words are "anarchism" and "as", as in the phrase "anarchism **originated** as". Hence, it is why for the word originated, our labels are "anarchism" and "as".

Similary, when the skip_window = 2 i.e we are considering the words 2 distance away as the labels, then for the word "as", the labels would be  2 words prior to "as" i.e  "anarchism" and "originated" and 2 words post to "as" i.e "a" and "term" as in the phrase "anarchism originated **as** a term"



Train a skip-gram model  with negative sampling
--------------------
**num_sampled ** : Number of negative  examples to sample i.e no of unrelatd word pairs to consider

In [9]:
print(batch)
print(labels)


[12 12 12 12  6  6  6  6]
[[2980]
 [ 181]
 [3181]
 [   6]
 [  12]
 [ 181]
 [   2]
 [2980]]


In [10]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

#with graph.as_default(), tf.device('/cpu:0'):
with graph.as_default():
  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
  print( 'train_dataset size is ', train_dataset.get_shape() ,' train_labels size is ', train_labels.get_shape() )
  print('embeddings has size [vocabulary_size, embedding_size] = [', vocabulary_size, embedding_size,']')
  # Variables.
  embeddings = tf.Variable( tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  softmax_weights = tf.Variable(  tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset)
  print('From embedding vector and train_dataset, input  embedding of  following size is computed', embed.get_shape())  
  print(embed)  
  # Compute the softmax loss, using a sample of the negative labels each time.
  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=embed,
                               labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

  # Optimizer.
  # Note: The optimizer will optimize the softmax_weights AND the embeddings.
  # This is because the embeddings are defined as a variable quantity and the
  # optimizer's `minimize` method will by default modify all variable quantities 
  # that contribute to the tensor it is passed.
  # See docs on `tf.train.Optimizer.minimize()` for more details.
  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset)
  similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

train_dataset size is  (128,)  train_labels size is  (128, 1)
embeddings has size [vocabulary_size, embedding_size] = [ 50000 128 ]
From embedding vector and train_dataset, input  embedding of  following size is computed (128, 128)
Tensor("embedding_lookup:0", shape=(128, 128), dtype=float32)


In [11]:
num_steps = 10001
#num_steps = 100001

with tf.Session(graph=graph) as session:
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
  print('Initialized')
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(
      batch_size, num_skips, skip_window)
    if(step == 0):
        print('1st Batch Training data  is ',batch_data)
        print('1st Batch Training label  is ',batch_data)
        print('1st Batch Training dataset size :',len(batch_data), ' Batch Training label size : ',len(batch_labels)  )
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = 'Nearest to %s:' % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = '%s %s,' % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

Initialized
1st Batch Training data  is  [   58    58   149   149   125   125   850   850   489   489  8335  8335
   133   133     1     1 35460 35460     2     2     1     1   115   115
  1016  1016     3     3     1     1 21303 21303     0     0     2     2
     1     1   175   175  1016  1016  3536  3536     1     1   181   181
    11    11   187   187    58    58     5     5     6     6 11754 11754
   212   212     7     7  1338  1338   102   102   426   426    20    20
    58    58  3237  3237   379   379     7     7  3782  3782     1     1
   796   796     2     2   399   399    28    28    41    41    37    37
    53    53   492   492    99    99    12    12     6     6  1422  1422
  3257  3257    17    17   565   565   779   779  4168  4168     1     1
   263   263  3181  3181    11    11   981   981]
1st Batch Training label  is  [   58    58   149   149   125   125   850   850   489   489  8335  8335
   133   133     1     1 35460 35460     2     2     1     1   115   115
  1

Final Word Vector Output
----------------------
**final_embeddings :** Final Word Vector Output is stored in the final_embeddings vector.
**[reverse_dictionary[i]** :  Labels of the word vector

In [55]:
#print(reverse_dictionary[1] )
#print(final_embeddings[1])
word_n_its_vector = { reverse_dictionary[i] :final_embeddings[i]  for i in range(len(reverse_dictionary)) } 
print( 'Sample of the word label and vector' )
print( { key : word_n_its_vector[key] for key in  word_n_its_vector.keys()[:2] } )
#print( word_n_its_vector[2] )        
#print( [word_n_its_vector[i] for i in range(3,5)] )


Sample of the word label and vector
{'fawn': array([-0.13989708,  0.12158715,  0.13733332, -0.12666136, -0.11127454,
        0.03098324,  0.08410942, -0.12983011, -0.02694082,  0.005929  ,
       -0.04977843,  0.04833758, -0.11463884,  0.11959696,  0.13467336,
        0.11713647,  0.02579618, -0.0366492 ,  0.10889158, -0.01245765,
        0.04406659, -0.0419971 , -0.1110612 ,  0.10188457, -0.09823871,
        0.13638397, -0.01920055,  0.09617165, -0.00555415,  0.13425246,
        0.09791777, -0.09968832,  0.00692748, -0.10811123,  0.10371564,
        0.13871436, -0.03038054, -0.09540132,  0.11353511, -0.12389319,
        0.07030345, -0.11490811, -0.01584335,  0.05678115,  0.09257349,
       -0.13135044, -0.09739818, -0.11123001,  0.06076265, -0.13553317,
       -0.09210048, -0.0321669 ,  0.04128449, -0.12536123, -0.09572752,
        0.02859585, -0.03789193, -0.03738993,  0.06962638, -0.11181191,
        0.04655303,  0.04477959,  0.12777688,  0.07807186,  0.08800848,
        0.0586958 ,

Finding Vector Simialrity between words :
-----------------------
**Case 1 :** he / she

**Case 2 :** he / originated

**Case 3 :** they / that    

In [54]:
from scipy import spatial
#print(type(word_n_its_vector['he']))
#print(word_n_its_vector['the'])
print('cosine distance similarity between  he /she is ',\
      1 - spatial.distance.cosine( word_n_its_vector['he'], word_n_its_vector['she']))
print('cosine distance similarity between  he /originated is ',\
      1 - spatial.distance.cosine( word_n_its_vector['he'], word_n_its_vector['originated']))
print('cosine distance similarity between  they vs that is ',\
      1 - spatial.distance.cosine( word_n_its_vector['they'], word_n_its_vector['that']))


cosine distance similarity between  he /she is  0.515669968902
cosine distance similarity between  he /originated is  -0.0176419312914
cosine distance similarity between  they vs that is  0.278272049324


Conclusion
--------------------
**"He vs She"** are very similar compared to  **"He vs Originated"**

In [12]:
num_points = 400

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points+1, :])

In [21]:
print(len(final_embeddings))
print(len(final_embeddings[0]))
print(final_embeddings[0])
#words = [reverse_dictionary[i] for i in range(1, num_points+1)]
print(reverse_dictionary[0], final_embeddings[0] )

50000
128
[-0.10473408 -0.04715362 -0.06638023  0.05636352  0.22023238  0.10935123
  0.21247007 -0.00649589 -0.0273759  -0.00662011 -0.11963457 -0.06362706
  0.20612746 -0.00543853 -0.04609589  0.01948957 -0.19230266  0.06368847
  0.08081017  0.01458507 -0.01219707  0.1040906  -0.00628894  0.00685745
 -0.02352546 -0.15802979  0.03076531 -0.0181194   0.11044451 -0.02193434
 -0.02483573 -0.16204436  0.0075853   0.02824622  0.05256828  0.11274089
  0.00669346  0.023488    0.04508581 -0.08424472 -0.07304773 -0.01650908
 -0.16038799  0.14383519  0.12079301  0.08757117  0.04898544  0.07272018
  0.01530478  0.04640829  0.01883251  0.19935384  0.10565738  0.00942819
  0.03475559  0.03978069 -0.07929466  0.0100785  -0.00998897  0.085611
  0.02543803  0.13312721 -0.10454167 -0.12847829 -0.029019    0.08624271
  0.06861328 -0.08197025  0.00885739 -0.04062694  0.00100554  0.06103877
  0.00572084  0.1667206  -0.02013417 -0.28820726 -0.04972165 -0.09361
  0.05357289  0.05451857  0.13149372  0.039497

In [28]:
def plot(embeddings, labels):
  assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
  pylab.figure(figsize=(15,15))  # in inches
  for i, label in enumerate(labels):
    x, y = embeddings[i,:]
    pylab.scatter(x, y)
    pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                   ha='right', va='bottom')
  pylab.show()

words = [reverse_dictionary[i] for i in range(1, num_points+1)]
plot(two_d_embeddings, words)

the
[ 0.04228502 -0.12075232 -0.11376955 -0.09859844  0.03488731  0.04238981
 -0.08082084  0.07563843 -0.33936015 -0.00882445  0.01562099  0.01937421
  0.1458983   0.10876398  0.05715398 -0.07455908  0.00659716 -0.00160836
 -0.036729   -0.01661832  0.0181931   0.05461675 -0.05124519 -0.01658517
 -0.08221611 -0.07063289  0.16708677 -0.00475863  0.05763826  0.02416596
 -0.0887311   0.0588704  -0.05295033 -0.09269209 -0.0109619  -0.00141115
  0.01545913 -0.03750875  0.05250677 -0.02560061 -0.0290512   0.12946072
 -0.08673239  0.07256972 -0.13782959 -0.07461096  0.07296053 -0.10163458
 -0.02597805 -0.08332466  0.03471394  0.08870099  0.03882752  0.01831625
 -0.17916873  0.11881664  0.06109766 -0.14970326 -0.11096389  0.02409375
 -0.11245652  0.13605402  0.07440074 -0.04972687  0.04708791 -0.14046784
  0.0202703  -0.01933543  0.04447813 -0.17779694  0.106643    0.11037834
 -0.0594922   0.0245506   0.06362081  0.08165073  0.09565093 -0.08075899
 -0.03995282  0.01146993 -0.04160945 -0.0942889

Conclusion 
------------------------
From  the above skip-gram model, we are able to train a model that given a text can automatically find the words similar to each other with skip-gram models.

In the process they are able to capture the relationshiop between words i.e. Dog, cat are alike (e.g such that they are both animals, four-legged, pets, etc).Also they are able to gather the semantic  relationship i.e Kitten is to cat what puppy is to dog.


---

Continous Bag of Words
-------

An alternative to skip-gram is another Word2Vec model called [CBOW](http://arxiv.org/abs/1301.3781) (Continuous Bag of Words). In the CBOW model, instead of predicting a context word from a word vector, you predict a word from the sum of all the word vectors in its context. We Implement and evaluate a CBOW model trained on the text8 dataset in the next project.

---